In [15]:
import gym
env = gym.make('MountainCar-v0')
print(env.reset())
print(env.step(env.action_space.sample()))
# print(env.observation_space.low)
env.close()

[-0.44971572  0.        ]
(array([-0.45126531, -0.0015496 ]), -1.0, False, {})


In [7]:
# Tiling
import random

def create_tiling_grid(low, high, bins=(10, 10), offsets=(0.0, 0.0)):
    assert len(low) == len(high) == len(bins) == len(offsets)
    
    dims = len(low)
    
    split_points = []
    for i in range(dims):
        step = (high[i] - low[i])/bins[i]
        
        split_points.append(np.linspace(low[i]+offsets[i]+step, high[i]+offsets[i], bins[i]-1, False))
                            
    return np.array(split_points)

def tile_encode(sample, high, low, tiling_specs, flatten=False):
    tilings = np.array([create_tiling_grid(low, high, tiling_specs[i][0], tiling_specs[i][1]) for i in range(len(tiling_specs))])
    encoded_sample = [[int(np.digitize(s, g)) for s, g in zip(sample, grid)] for grid in tilings]
    features = []
    for s, spec in zip(encoded_sample, tiling_specs):
        f = np.zeros(spec[0])
        f[tuple(s)] = 1
        features.append(f.flatten())
        
#     print(features)
    return np.concatenate(features)
#     one_hot = [[1 if i == index else 0 for index, bins in zip(sample, spec[0]) for i in range(bins)] for sample, spec in zip(encoded_sample, tiling_specs)]    
#     return np.concatenate(one_hot) if flatten else one_hot

def action_value_delta(state, action, weights, high, low, tiling_specs):
    state_action = (state[0], state[1], action)
    return tile_encode(state_action, high, low, tiling_specs, True)

def action_value_approx(state, action, weights, high, low, tiling_specs):
    # State = (float, float)
    # Action = 0 <= int <= 2
    state_action = (state[0], state[1], action)
    x = tile_encode(state_action, high, low, tiling_specs, True)
    return x * weights

def epsilon_greedy(state, actions, Q, weights, high, low, tiling_specs, epsilon):
    if random.random() < epsilon:
        return random.choice(actions)
    else :
        action_values = [(action, sum(Q(state, action, weights, high, low, tiling_specs))) for action in actions]
        max_action_value = None
        max_actions = []
        for a, v in action_values:
            if max_action_value == None or v > max_action_value:
                max_action_value = v
                max_actions = [a]
            elif v == max_action_value:
                max_actions.append(a)
#         print(action_values)
        return random.choice(max_actions)

# Test with some sample values
# samples = [(-0.2 , 0.067, 1)]
# TILINGS = 8
# tiling_specs = [((TILINGS, TILINGS, 3), (-0.15, -0.015, 0)),
#             ((TILINGS, TILINGS, 3), (0.0, 0.0, 0)),
#             ((TILINGS, TILINGS, 3), (0.15, 0.015, 0))]
# low = [-1.2,  -0.07, 0]
# high = [0.6, 0.07, 2]
# encoded_samples = [tile_encode(sample, high, low, tiling_specs, True) for sample in samples]
# print("\nSamples:", repr(samples), sep="\n")
# print("\nEncoded samples:", repr(encoded_samples), sep="\n")

In [32]:
import numpy as np
import gym
from operator import mul
from tqdm import tqdm

env = gym.make('MountainCar-v0')
env._max_episode_steps = 2000

EPISODES = 1000
NUM_LOGS = 10

Q = action_value_approx
DELTA_Q = action_value_delta
POLICY = epsilon_greedy

ALPHA = 0.5 / 4
EPSILON = 0
GAMMA = 1
ACTIONS = [0, 1, 2]

TILES = (8, 8, 3)
TILINGS = 8
MIN_TILE_OFFSET = (-0.15, -0.015, 0)
MAX_TILE_OFFSET = (0.15, 0.015, 0)
LOW = (-1.2, -0.07, 0)
HIGH = (1.2, 0.07, 2)

w = np.zeros(np.prod(TILES) * TILINGS)
print(w.shape)
tiling_specs = [(TILES, tuple(min_off + (max_off - min_off)*i/(TILINGS-1 if TILINGS > 1 else 1)
                                          for min_off, max_off in zip(MIN_TILE_OFFSET, MAX_TILE_OFFSET))) 
                for i in range(TILINGS)]

total_reward = 0


for e in tqdm(range(EPISODES)):
    s = env.reset()
    a = POLICY(s, ACTIONS, Q, w, HIGH, LOW, tiling_specs, EPSILON)
    done = False
    episode_reward = 0
    step = 0
    while not done:
#         env.render()
        s_prime, r, done, info = env.step(a)
#         done = False
        if done:
            w = w + ALPHA * (r - Q(s, a, w, HIGH, LOW, tiling_specs)) * DELTA_Q(s, a, w, HIGH, LOW, tiling_specs)
        else:
            a_prime = POLICY(s_prime, ACTIONS, Q, w, HIGH, LOW, tiling_specs, EPSILON)
            w = w + ALPHA * (r + GAMMA * sum(Q(s_prime, a_prime, w, HIGH, LOW, tiling_specs)) - sum(Q(s, a, w, HIGH, LOW, tiling_specs))) * DELTA_Q(s, a, w, HIGH, LOW, tiling_specs)
            s = s_prime
            a = a_prime
        
        episode_reward += r
        total_reward += r
        step += 1

        if r != -1:
            print(r)
            print(step)
        
    if e % (EPISODES // NUM_LOGS) == 0:
        print("Episode {} with reward {}. Average reward: {}".format(e, episode_reward, total_reward / (EPISODES // NUM_LOGS)))
        total_reward = 0




  0%|          | 0/1000 [00:00<?, ?it/s]

(1536,)




  0%|          | 1/1000 [00:02<44:45,  2.69s/it]

Episode 0 with reward -822.0. Average reward: -8.22




  0%|          | 2/1000 [00:05<42:54,  2.58s/it]

  0%|          | 3/1000 [00:06<37:01,  2.23s/it]

  0%|          | 4/1000 [00:08<34:08,  2.06s/it]

  0%|          | 5/1000 [00:08<27:06,  1.63s/it]

  1%|          | 6/1000 [00:10<25:39,  1.55s/it]

  1%|          | 7/1000 [00:10<22:24,  1.35s/it]

  1%|          | 8/1000 [00:12<21:22,  1.29s/it]

  1%|          | 9/1000 [00:13<20:25,  1.24s/it]

  1%|          | 10/1000 [00:14<19:06,  1.16s/it]

  1%|          | 11/1000 [00:15<20:12,  1.23s/it]

  1%|          | 12/1000 [00:16<18:22,  1.12s/it]

  1%|▏         | 13/1000 [00:16<15:24,  1.07it/s]

  1%|▏         | 14/1000 [00:17<13:17,  1.24it/s]

  2%|▏         | 15/1000 [00:18<14:31,  1.13it/s]

  2%|▏         | 16/1000 [00:19<13:02,  1.26it/s]

  2%|▏         | 17/1000 [00:19<13:11,  1.24it/s]

  2%|▏         | 18/1000 [00:20<13:05,  1.25it/s]

  2%|▏         | 19/1000 [00:21<12:56,  1.26it/s]

  2%|▏         | 20/1000 [00:22<12:47,  1.28it/s]

  2%|▏         | 21/1000 [00:22<11:54

...



 70%|███████   | 702/1000 [06:04<02:25,  2.05it/s]

 70%|███████   | 703/1000 [06:05<02:22,  2.09it/s]

 70%|███████   | 704/1000 [06:05<02:30,  1.97it/s]

 70%|███████   | 705/1000 [06:06<02:28,  1.98it/s]

 71%|███████   | 706/1000 [06:06<02:21,  2.07it/s]

 71%|███████   | 707/1000 [06:07<02:20,  2.08it/s]

 71%|███████   | 708/1000 [06:07<02:25,  2.01it/s]

 71%|███████   | 709/1000 [06:07<02:15,  2.15it/s]

 71%|███████   | 710/1000 [06:08<02:06,  2.29it/s]

 71%|███████   | 711/1000 [06:08<02:01,  2.38it/s]

 71%|███████   | 712/1000 [06:09<01:59,  2.41it/s]

 71%|███████▏  | 713/1000 [06:09<01:53,  2.52it/s]

 71%|███████▏  | 714/1000 [06:09<01:50,  2.59it/s]

 72%|███████▏  | 715/1000 [06:10<01:46,  2.69it/s]

 72%|███████▏  | 716/1000 [06:10<01:44,  2.71it/s]

 72%|███████▏  | 717/1000 [06:10<01:42,  2.77it/s]

 72%|███████▏  | 718/1000 [06:11<01:39,  2.83it/s]

 72%|███████▏  | 719/1000 [06:11<01:40,  2.81it/s]

 72%|███████▏  | 720/1000 [06:11<01:34,  2.96it/s]

 72%|█████

Episode 800 with reward -160.0. Average reward: -140.98




 80%|████████  | 802/1000 [06:52<01:48,  1.83it/s]

 80%|████████  | 803/1000 [06:52<01:43,  1.90it/s]

 80%|████████  | 804/1000 [06:53<01:41,  1.94it/s]

 80%|████████  | 805/1000 [06:53<01:44,  1.86it/s]

 81%|████████  | 806/1000 [06:54<01:46,  1.81it/s]

 81%|████████  | 807/1000 [06:54<01:42,  1.88it/s]

 81%|████████  | 808/1000 [06:55<01:43,  1.86it/s]

 81%|████████  | 809/1000 [06:56<01:41,  1.88it/s]

 81%|████████  | 810/1000 [06:56<01:38,  1.94it/s]

 81%|████████  | 811/1000 [06:56<01:28,  2.13it/s]

 81%|████████  | 812/1000 [06:57<01:28,  2.12it/s]

 81%|████████▏ | 813/1000 [06:57<01:32,  2.03it/s]

 81%|████████▏ | 814/1000 [06:58<01:27,  2.13it/s]

 82%|████████▏ | 815/1000 [06:58<01:20,  2.29it/s]

 82%|████████▏ | 816/1000 [06:58<01:15,  2.42it/s]

 82%|████████▏ | 817/1000 [06:59<01:07,  2.70it/s]

 82%|████████▏ | 818/1000 [06:59<01:06,  2.75it/s]

 82%|████████▏ | 819/1000 [06:59<01:05,  2.77it/s]

 82%|████████▏ | 820/1000 [07:00<01:04,  2.81it/s]

 82%|█████

Episode 900 with reward -137.0. Average reward: -125.88




 90%|█████████ | 902/1000 [07:35<00:45,  2.16it/s]

 90%|█████████ | 903/1000 [07:36<00:48,  2.00it/s]

 90%|█████████ | 904/1000 [07:36<00:50,  1.92it/s]

 90%|█████████ | 905/1000 [07:37<00:49,  1.92it/s]

 91%|█████████ | 906/1000 [07:37<00:49,  1.91it/s]

 91%|█████████ | 907/1000 [07:38<00:43,  2.12it/s]

 91%|█████████ | 908/1000 [07:38<00:40,  2.25it/s]

 91%|█████████ | 909/1000 [07:39<00:41,  2.19it/s]

 91%|█████████ | 910/1000 [07:39<00:39,  2.31it/s]

 91%|█████████ | 911/1000 [07:40<00:41,  2.14it/s]

 91%|█████████ | 912/1000 [07:40<00:38,  2.29it/s]

 91%|█████████▏| 913/1000 [07:40<00:36,  2.41it/s]

 91%|█████████▏| 914/1000 [07:41<00:39,  2.18it/s]

 92%|█████████▏| 915/1000 [07:41<00:40,  2.11it/s]

 92%|█████████▏| 916/1000 [07:42<00:36,  2.27it/s]

 92%|█████████▏| 917/1000 [07:42<00:37,  2.21it/s]

 92%|█████████▏| 918/1000 [07:43<00:34,  2.35it/s]

 92%|█████████▏| 919/1000 [07:43<00:32,  2.46it/s]

 92%|█████████▏| 920/1000 [07:43<00:29,  2.67it/s]

 92%|█████

In [35]:
val_file_name = "grad-sarsa-out/mc"
import pickle
with open("{}-weights.pickle".format(val_file_name), 'wb') as handle:
    pickle.dump(w, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

In [38]:
sum(w_save)

-8848.486284904206

In [9]:
from math import floor

def hash_coords(coordinates, m, read_only=False):
    if isinstance(m, int): return hash(tuple(coordinates)) % m
    if m is None: return coordinates

def tiles(iht_or_size, num_tilings, floats, ints=None, read_only=False):
    """returns num-tilings tile indices corresponding to the floats and ints"""
    if ints is None:
        ints = []
    qfloats = [floor(f * num_tilings) for f in floats]
    tiles = []
    for tiling in range(num_tilings):
        tilingX2 = tiling * 2
        coords = [tiling]
        b = tiling
        for q in qfloats:
            coords.append((q + b) // num_tilings)
            b += tilingX2
        coords.extend(ints)
        tiles.append(hash_coords(coords, iht_or_size, read_only))
    return tiles

tiles(10, 2, [1.2, 0.02], [1])

[1, 9]